In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Definindo transformações para o conjunto de dados
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Carregando e preparando os dados CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# Definindo a CNN personalizada
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # 1st Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 2nd Convolutional layer
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)  # 3rd Convolutional layer
        self.fc1 = nn.Linear(64 * 4 * 4, 10)  # Fully connected layer

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc1(x)
        return x

# Instanciando a CNN customizada
custom_model = CustomCNN()

# Definindo a função de perda e o otimizador para a CNN personalizada
criterion = nn.CrossEntropyLoss()
optimizer_custom = optim.SGD(custom_model.parameters(), lr=0.001, momentum=0.9)

# Treinando a CNN personalizada
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer_custom.zero_grad()
        outputs = custom_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_custom.step()
        running_loss += loss.item()
        if i % 200 == 199:    # Imprimir a cada 200 mini-batches
            print(f'Epoch [{epoch + 1}, {i + 1}] Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training Custom CNN')

# Transfer Learning usando ResNet-18
resnet_model = models.resnet18(pretrained=True)

# Congelando todos os parâmetros, exceto os da camada FC
for param in resnet_model.parameters():
    param.requires_grad = False

# Substituindo a camada FC para ajustar à saída da CIFAR-10 (10 classes)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 10)

# Definindo a função de perda e o otimizador para a transfer learning
optimizer_resnet = optim.SGD(resnet_model.fc.parameters(), lr=0.001, momentum=0.9)

# Treinando apenas a camada FC da ResNet-18 por 1 época
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer_resnet.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_resnet.step()
        running_loss += loss.item()
        if i % 200 == 199:    # Imprimir a cada 200 mini-batches
            print(f'Epoch [{epoch + 1}, {i + 1}] Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Transfer Learning with ResNet-18')

# Função para calcular a acurácia
def get_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total) * 100

# Obtendo a acurácia para ambas as redes
accuracy_custom = get_accuracy(custom_model, testloader)
accuracy_resnet = get_accuracy(resnet_model, testloader)

print(f'Acurácia da CNN customizada: {accuracy_custom:.2f}%')
print(f'Acurácia da ResNet-18 com transfer learning: {accuracy_resnet:.2f}%')


100%|██████████| 170498071/170498071 [05:47<00:00, 490413.89it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1, 200] Loss: 2.300
Epoch [2, 200] Loss: 2.147
Epoch [3, 200] Loss: 1.864
Epoch [4, 200] Loss: 1.738
Epoch [5, 200] Loss: 1.622
Epoch [6, 200] Loss: 1.532
Epoch [7, 200] Loss: 1.461
Epoch [8, 200] Loss: 1.419
Epoch [9, 200] Loss: 1.379
Epoch [10, 200] Loss: 1.345
Epoch [11, 200] Loss: 1.307
Epoch [12, 200] Loss: 1.267
Epoch [13, 200] Loss: 1.249
Epoch [14, 200] Loss: 1.227
Epoch [15, 200] Loss: 1.193
Epoch [16, 200] Loss: 1.171
Epoch [17, 200] Loss: 1.143
Epoch [18, 200] Loss: 1.128
Epoch [19, 200] Loss: 1.107
Epoch [20, 200] Loss: 1.086
Finished Training Custom CNN


c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Pichau/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:16<00:00, 2.76MB/s]


Epoch [1, 200] Loss: 1.998
Finished Transfer Learning with ResNet-18
Acurácia da CNN customizada: 60.98%
Acurácia da ResNet-18 com transfer learning: 40.96%
